In [1]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import json
import os

path = r'C:\Users\ruanv\OneDrive\Documentos\Mestrado\Projeto de Mestrado\bases'

df = pd.read_csv(os.path.join(path,'demandaTrafos_2018-2023.csv'), sep=';', encoding='latin-1')

with open(os.path.join(path,'schema_trafos.json'), 'r', encoding='utf-8') as file:
    schema = json.load(file)

In [2]:


df['Data/Hora Medição'] = pd.to_datetime(df['Data/Hora Medição'], dayfirst=True, errors='coerce')
df['ANO'] = df['Data/Hora Medição'].dt.year
df['MES'] = df['Data/Hora Medição'].dt.month
df['Dia_Mes_Hora'] = df['Data/Hora Medição'].dt.strftime('%d/%m %H:%M')

# Garantindo que 'Dia_Mes_Hora' seja uma string e sem valores nulos antes de aplicar a condição
df = df[pd.notna(df['Dia_Mes_Hora'])]

# Filtrar as linhas que não começam com '29/02', pois os anos bissextos bagunçam o gráfico
df = df[~df['Dia_Mes_Hora'].str.startswith('29/02')]

# Calculando a Potência Aparente e o Fator de Potência
df['Potencia Aparente'] = np.sqrt(df['Potencia Ativa']**2 + df['Potencia Reativa']**2)
df['Fator de Potência'] = np.where(df['Potencia Aparente'] != 0, df['Potencia Ativa'] / df['Potencia Aparente'], np.nan)

df = df.astype(schema)

df = df[df['Potencia Aparente'] != 0]

# Agrupar dados por TRAFO, ANO e Dia_Mes_Hora, ordenando corretamente pelo tempo
grouped = df.groupby(['TRAFO', 'ANO', 'Data/Hora Medição'])['Potencia Aparente'].mean().reset_index()

grouped['Dia_Mes_Hora'] = grouped['Data/Hora Medição'].dt.strftime('%d/%m %H:%M')

# Lista de trafos das subestações da PB
trafos = grouped['TRAFO'].unique()

In [ ]:
# Plot do gráfico

fig = go.Figure()

# Adicionar linhas para cada TRAFO e ANO
for trafo in trafos:
    df_trafo = grouped[grouped['TRAFO'] == trafo]
    potencia_nominal = df[df['TRAFO'] == trafo]['POTENCIA (MVA)'].iloc[0]
    
    # Adicionar a linha de Potência Nominal
    fig.add_trace(go.Scatter(
        x=df_trafo['Dia_Mes_Hora'],
        y=[potencia_nominal] * len(df_trafo),  # Linha horizontal com valor fixo da potência nominal
        mode='lines',
        name=f'POTÊNCIA NOMINAL TRAFO: {trafo}',
        line=dict(dash='dash', color='red'),  # Linha tracejada vermelha para a potência nominal
        visible=(trafo == trafos[0])  # Mostrar apenas o primeiro TRAFO por padrão
    ))

    for ano in df_trafo['ANO'].unique():
        df_ano = df_trafo[df_trafo['ANO'] == ano]
        fig.add_trace(go.Scatter(
            x=df_ano['Dia_Mes_Hora'],
            y=df_ano['Potencia Aparente'],
            mode='lines',
            name=f'TRAFO: {trafo} - Ano: {ano}',
            visible=(trafo == trafos[0])  # Mostrar apenas o primeiro TRAFO por padrão
        ))

# Configurar lista suspensa para selecionar o TRAFO
dropdown_buttons = [
    {
        'label': trafo, 
        'method': 'update', 
        'args': [
            {'visible': [
                (trafo == trace.name.split(' - ')[0].split(': ')[1]) 
                or (f'POTÊNCIA NOMINAL TRAFO: {trafo}' == trace.name)
                for trace in fig.data
            ]},
            {'title': f'TRAFO: {trafo}'}
        ]
    }
    for trafo in trafos
]

# Configurar checkboxes para selecionar os ANOS
year_buttons = [
    {
        'label': str(ano),
        'method': 'update',
        'args': [
            {
                'visible': [
                    (
                        len(trace.name.split(' - ')) > 1 and  # Certifica que há pelo menos um ' - ' no nome
                        len(trace.name.split(' - ')[1].split(': ')) > 1 and  # Certifica que há um ': ' após o ' - '
                        ano == int(trace.name.split(' - ')[1].split(': ')[1])
                    )
                    if 'TRAFO' in trace.name else False
                    for trace in fig.data
                ]
            }
        ],
    }
    for ano in grouped['ANO'].unique()
]

# Adicionar os menus de dropdown e checkboxes ao layout
fig.update_layout(
    updatemenus=[
        {
            'active': 0,
            'buttons': dropdown_buttons,
            'direction': 'down',
            'showactive': True,
            'x': 0.1,
            'xanchor': 'left',
            'y': 1.15,
            'yanchor': 'top'
        }
    ],
    showlegend=True,
    legend=dict(
        title='Anos',
        orientation='h',
        x=0.1,
        xanchor='left',
        y=1.05,
        yanchor='top'
    ),
)

# Adicionar checkboxes para os anos no gráfico
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            buttons=year_buttons,
            direction="down",
            x=0.3,
            xanchor="left",
            y=1.15,
            yanchor="top",
            showactive=True,
            active=-1,
        ),
        dict(
            type="dropdown",
            buttons=dropdown_buttons,
            x=0.1,
            xanchor="left",
            y=1.15,
            yanchor="top",
        )
    ]
)

fig.show()
